In [2]:
 import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [3]:
hr = pd.read_csv('HR Analytics.csv')
hr_dummies=pd.get_dummies(hr)
hr_dummies.head()
train,test =train_test_split(hr_dummies,test_size=0.3,random_state=100)
train_y =train['Attrition']
test_y =test['Attrition']

train_x=train.drop('Attrition',axis=1)
test_x=test.drop('Attrition',axis=1)

from sklearn.ensemble import RandomForestClassifier
model =RandomForestClassifier(random_state=100)
model.fit(train_x,train_y)

test_pred =model.predict(test_x)
df_pred =pd.DataFrame({'actual':test_y,'predicted':test_pred})
df_pred['pred_status']=df_pred['actual'] == df_pred['predicted']
df_pred[df_pred['pred_status']==True].shape[0]/df_pred.shape[0] *100

85.71428571428571

#confusion matrix
- True positive
- True Negative
- False Positive
- False Negative
- confusion matrix

- Predicted Actual
- 0,          0 -  TN
- 0,          1 -  FN
- 1,          0 -  FP
- 1,          1 -  TP

In [4]:
df_pred.head()

,actual,predicted,pred_status
880,0,0,True
152,0,0,True
1466,0,0,True
1084,0,0,True
1086,0,0,True


In [5]:
df_pred['status']=np.where((df_pred['actual']==0) & (df_pred['predicted']==1) ,'FP',0)

In [6]:
df_pred[df_pred['status']=='FP'].shape[0]

4

In [8]:
df_pred=df_pred.drop('status',axis=1)

In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df_pred['actual'],df_pred['predicted'])

array([[367,   4],
       [ 59,  11]], dtype=int64)

In [10]:
tn,fp,fn,tp = confusion_matrix(df_pred['actual'],df_pred['predicted']).ravel()
print(tn,fp,fn,tp)

367 4 59 11


- accuracy in confusion matrix = tf+tn/tf+tn+fp+fn
- sensitivity = tp/tp+fn = % of positive correct prediction
- my ability to predict 1's as 1 is called sensitivity
- my ability to predict 0's as 0 is called specificity
- specificity = tn /fp+tn = % of negative correct prediction
- inbalanced class will have specificity more but sensitivity more

# Decision Tree 

In [44]:
model =DecisionTreeClassifier(random_state=100)
model.fit(train_x,train_y)

test_pred =model.predict(test_x)
df_pred =pd.DataFrame({'actual':test_y,'predicted':test_pred})
df_pred['pred_status']=df_pred['actual'] == df_pred['predicted']
df_pred[df_pred['pred_status']==True].shape[0]/df_pred.shape[0] *100

78.00453514739229

In [49]:
tp =df_pred[(df_pred['predicted']==1) &(df_pred['actual']==1)].shape[0]
tn =df_pred[(df_pred['predicted']==0) &(df_pred['actual']==0)].shape[0]
fp =df_pred[(df_pred['predicted']==1) &(df_pred['actual']==0)].shape[0]
fn=df_pred[(df_pred['predicted']==0) &(df_pred['actual']==1)].shape[0]
print("true positive:%d" %tp)
print("true negative:%d" %tn)
print("false positive:%d" %fp)
print("false negative:%d" %fn)

true positive:32
true negative:312
false positive:59
false negative:38


In [50]:
tn,fp,fn,tp = confusion_matrix(df_pred['actual'],df_pred['predicted']).ravel()
print(tn,fp,fn,tp)

312 59 38 32


In [46]:
#Accuracy
(312+32)/(441)

0.780045351473923

In [47]:
#Sensitivity =tp/tp+fn
32/(32+38)

0.45714285714285713

In [48]:
#Specificity =tn/tn+fp
312/(312+59)

0.8409703504043127

In [1]:
32/(32+59)

0.3516483516483517

In [52]:
from sklearn.metrics import classification_report
print(classification_report(df_pred['actual'],df_pred['predicted']))

             precision    recall  f1-score   support

          0       0.89      0.84      0.87       371
          1       0.35      0.46      0.40        70

avg / total       0.81      0.78      0.79       441



# Adaptive Boosting

In [11]:
from sklearn.ensemble import AdaBoostClassifier

In [12]:
model = AdaBoostClassifier(random_state=100)
model.fit(train_x,train_y)

test_pred =model.predict(test_x)
tn,fp,fn,tp = confusion_matrix(test_y,test_pred).ravel()
tn,fp,fn,tp

(352, 19, 43, 27)

In [13]:
#accuracy
ab_accuracy=(352+27)/(352+43+19+27) *100

In [14]:
#sensitivity
ab_sensitivity=tp/(tp+fn) * 100
ab_sensitivity

38.57142857142858

In [15]:
#specificity
ab_specificity =tn /(tn+fp) *100
ab_specificity

94.87870619946092

# Weighted Sampling

In [30]:
import pandas as pd
import numpy as np
pd.Series(np.random.randint(1,100,100)).value_counts().shape[0]

60

In [17]:
random_samples=[]
for i in range(100):
    random_samples.append(np.random.choice([1,2,3],p=[0,0.2,0.8]))
pd.Series(random_samples).value_counts()

3    80
2    20
dtype: int64

# Classification  Error Rate

In [21]:
samples_dt1 =train.iloc[np.random.randint(1,train.shape[0],train.shape[0])]
all_cols =np.array(train.drop('Attrition',axis=1).columns)
cols_position = np.random.randint(1,len(all_cols),3)
random_cols = all_cols[cols_position]
samples_dt1.shape
samples_dt1_x =samples_dt1.drop('Attrition',axis=1)[random_cols]
samples_dt1_y =samples_dt1['Attrition']

In [19]:
pd.Series(samples_dt1.index).value_counts().head()

1158    5
835     5
1437    4
983     4
337     4
dtype: int64

In [22]:
dt1 =DecisionTreeClassifier(max_depth=1)
dt1.fit(samples_dt1_x,samples_dt1_y)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [23]:
test_pred= dt1.predict(test_x[random_cols])


In [24]:
df_pred =pd.DataFrame({'actual':test_y,'predicted':test_pred})
df_pred.head()

,actual,predicted
880,0,0
152,0,0
1466,0,0
1084,0,0
1086,0,0


In [25]:
tn,fp,fn,tp = confusion_matrix(test_y,test_pred).ravel()
tn,fp,fn,tp


(371, 0, 70, 0)

 - positive -1 negative -1
 - DT1- random rows and random cols
  - depth=1 ,with equal probablities
 - predict for train data
 - Et = no of wrong predictions/total no of predictions
 - dt = 1/2 log(1-et/et)
 - calculate probablity for each sample on train data
 - new probablity for each sample = old probability * exp(-alphat *  actual value * predicted value) / z
 - D2 - sample rows with unequal probabilities,random columns - decision stump
 - repeat the process for new trees
 - Error rate = fp+fn /(tp+tn+fp+fn)
 - Et=1- bad classifier  : alphat - negative
 - Et=0.5-average classifier: aplhat - zero 
 - Et=0- good classifier:alphat-positive
  dt+1= (dt(1)) * exponential(alphat * yt(i) * ht(i)/z 
  probablitiy of sample   y           predicted     new probability (exp(-alphat * actual value * predicted))
     1/1029             1(positive)  1(positive)          0.00079/z      E(-alphat)  
     1/1029             1(positive)  0(negative)          0.0011/z       E(alphat)
     

# - Differences between Random Forest and Adaptive Boosting
  - Random Forest
    - Trees are independent on each other
    - can run on parallel
    - row samples with equal probability
    - full tree can be created
  - Adaptive Boosting
    - Trees are dependent on each other
    - sequential training
    - rows are sampled with unequal probability
    - decision trees are created(depth=1)
    - Each decision tree is a weak learner
    - combine weak learners to generate strong learner
    

In [293]:
Et =fp+fn/(tp+tn+fp+fn)
Et


14.151927437641723

In [300]:
import math
alphat = 1/2 * (math.log((1-0.158)/0.158))
alphat

0.8365924906076799